In [1]:
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
import json

In [2]:
class firefighterProblem:
    def __init__(self,fire_file, firefighter_file, node_file):
        #the section can be modified
        self.T_number = 150 #時間長度
        self.T = list([i for i in range(self.T_number + 1)]) #時間list
        self.P = {1:3,2:4} #各個消防單位時間處理的燃料量
        self.mode = "not random"
        
        self.model = gp.Model("FIREFIGHTER")
        self.M = 10
        self.epsilon=1e-4
        self.A_p = gp.tuplelist()
        self.A_f = gp.tuplelist()
        self.tau = gp.tupledict()
        self.lamb = gp.tupledict()
        self.K = set() #K=消防員集合
        self.Q = {}
        self.b = {}
        self.H = {}
        self.process = {}
        self.A_f_NEIGHBOR = {} #A_f_NEIGHBOR=與點i相鄰的點
        self.A_f_NEIGHBOR_T = {} #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點
        self.x = {}
        self.w = {}
        self.u = {}
        self.u_bar = {}
        self.v = {}
        self.v_bar = {}
        self.NODE_POS = {}
        
        self.read_from_excel(node_file)
        self.read_from_excel(fire_file)
        self.read_from_excel(firefighter_file)
        
    def read_from_excel(self, fileName):
        if "fire_route" in fileName:
            fire_df = pd.read_excel(fileName)
            df_num = len(fire_df.index)
            for i in range(df_num):
                u = int(fire_df.iloc[i]['i'])
                v = int(fire_df.iloc[i]['j'])
                time = fire_df.iloc[i]['travel time']
                self.lamb[u,v] = time
                self.A_f.append((u,v))
        elif "firefighter_route" in fileName:
            firefighter_df = pd.read_excel(fileName)
            df_num = len(firefighter_df.index)
            for i in self.N:
                self.A_p.append((i,i))
            for i in range(df_num):
                u = int(firefighter_df.iloc[i]['i'])
                v = int(firefighter_df.iloc[i]['j'])
                firefighterIndex = firefighter_df.iloc[i]['k']
                if firefighterIndex not in self.K:
                    self.K.add(int(firefighterIndex))
                time = firefighter_df.iloc[i]['travel time']
                self.tau[u,v,firefighterIndex] = time
                #避免在多消防員時重複紀錄arc set
                if (u,v) not in self.A_p:
                    self.A_p.append((u,v))  
        elif "nodeInformation" in fileName:
            nodeInfo1 = pd.read_excel(fileName, 'coordinates')
            df_num = len(nodeInfo1.index)
            self.N_number = df_num
            self.N = set([i for i in range(1, self.N_number+1)])
            
            for i in range(self.N_number):
                self.NODE_POS[i+1] = (int(nodeInfo1.iloc[i]['x']), int(nodeInfo1.iloc[i]['y']))
            if self.mode == 'random':
                for i in range(self.N_number):
                    self.Q[i+1] = random.randint(2,5)
                    self.b[i+1] = random.randint(20,30)
                    self.H[i+1] = random.randint(2,5)
            else:
                for i in range(self.N_number):
                    self.Q[i+1] = int(nodeInfo1.iloc[i]['quantity'])
                    self.b[i+1] = int(nodeInfo1.iloc[i]['value'])
                    self.H[i+1] = int(nodeInfo1.iloc[i]['burning time'])
            
            nodeInfo2 = pd.read_excel(fileName, 'source')
            self.N_D = set(nodeInfo2['N_D'].tolist())
            self.N_F = set(nodeInfo2['N_F'].tolist())
    def initialize(self):
        for k in self.K:
            self.process[k]={}
            for i in self.N:
                if i in self.N_D:
                    self.process[k][i] = 0
                else:
                    self.process[k][i] = math.ceil(self.Q[i] * self.H[i] / self.P[k])
        for l in self.N - self.N_D:                          #定義A_f_NEIGHBOR
            connect = self.A_f.select('*',l)
            self.A_f_NEIGHBOR[l]=[]
            for temp in connect:
                self.A_f_NEIGHBOR[l].append(temp[0])
        for j in self.N - self.N_D - self.N_F:                         #定義A_f_NEIGHBOR_T
            for t in self.T:
                self.A_f_NEIGHBOR_T[j, t]=[]
                for i in self.A_f_NEIGHBOR[j]:
                    if t - self.H[i] - self.lamb[i, j]>=0:
                        self.A_f_NEIGHBOR_T[j, t].append(i)
        #定義x[i,j,k,t]
        for k in self.K:
            for t in self.T:
                for i in range(len(self.A_p)):
                    self.x[self.A_p[i][0], self.A_p[i][1], k, t] = self.model.addVar(vtype='B', name="x[%d,%d,%d,%d]" % (self.A_p[i][0], self.A_p[i][1], k, t))
        #定義w[i,k,t]
        for k in self.K:
            for t in self.T:
                for i in self.N:
                    self.w[i,k,t] = self.model.addVar(vtype='B',name="w[%d,%d,%d]" % (i, k, t))

        #定義u[i,t]
        self.u = self.model.addVars(self.N, self.T, vtype="B", name="u")

        #定義u_bar[i,k,t]
        self.u_bar = self.model.addVars(self.N, self.K, self.T, vtype="B", name="u_bar")

        #定義v[i,t]
        self.v = self.model.addVars(self.N, self.T, vtype="B", name="v")
        
        #定義v_bar[i,t]
        self.v_bar = self.model.addVars(self.N, self.T, vtype="B", name="v_bar")        
        
        self.model.update()
        
        #原點flow blance
        for k in self.K:                        
            self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i,j in self.A_p) <= 1)
    
        #限定從depot出發
        for O in self.N_D:
            connect = self.A_p.select(O,'*')
            for k in self.K:
                self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i, j in connect) == 1)
                
        #depot不會被保護
        #self.model.addConstrs((self.u_bar[i,k,t]==0 for i in self.N_D for k in self.K for t in range(self.T_number+1)))

        #flow balance
        for k in self.K:
            for t in range(1,self.T_number):
                for j in self.N: 
                    in_connect = self.A_p.select('*',j)
                    out_connect = self.A_p.select(j,'*')
                    temp = 0 #in-degree
                    temp += self.w[j, k, t - 1] # t-1在j idle
                    
                    if j in self.N_D: #j in depot set會有u_bar，只是都為0
                        temp += self.u_bar[j, k, t]
                    else: #j not in depot set, 若現在的t > process time，則有u_bar且為非0
                        if self.process[k][j] <= t:
                            temp += self.u_bar[j, k, t - self.process[k][j]]
                    for m, n in in_connect:
                        if m != n and self.tau[m, n, k] <= t: #若現在的t>travel time，則會有x
                            temp += self.x[m, n, k, t - self.tau[m,n,k]]
                    self.model.addConstr(temp == gp.quicksum(self.x[n, w, k, t] for n, w in out_connect), name="flow") #in-degree = out-degree
        
        #若在t from i to i(i not include depot), 一定會是在t時刻開始保護或idle
        self.model.addConstrs((self.u_bar[i, k, t] + self.w[i, k, t] == self.x[i, i, k, t] for i in self.N - self.N_D for k in self.K for t in range(self.T_number)))
        
        #若在t from s to s(s is in deopt set), 一定會在t時刻idle
        self.model.addConstrs((self.w[s, k, t] == self.x[s, s, k, t]) for s in self.N_D for k in self.K for t in range(self.T_number))
        
        #每個node在T內只會開始燒、開始保護、或未影響
        self.model.addConstrs(self.u[i, t] + self.u_bar.sum(i, '*', t) <= 1 for i in self.N for t in self.T)
        
        #開始燒與已經燒之間的關係
        self.model.addConstrs(self.v[i, t] + self.u[i, t] == self.v[i, t+1] for i in self.N for t in self.T[0:-1])
        
        #開始保護與已經保護之間的關係
        self.model.addConstrs(self.v_bar[i, t] + self.u_bar.sum(i, '*', t) == self.v_bar[i, t+1] for i in self.N for t in self.T[0:-1]) #constrain 9
        
        #deopt在T內不會開始保護
        self.model.addConstrs(self.u_bar.sum(s, '*', '*') == 0 for s in self.N_D)
        
        #depot在T內不會開始燒
        self.model.addConstrs(self.u.sum(s, '*') == 0 for s in self.N_D)
        
        #火焰的延燒
        for j in self.N - self.N_D - self.N_F:
            for t in range(self.T_number):
                if len(self.A_f_NEIGHBOR_T[j, t]) == 0:
                    self.model.addConstr(self.u[j, t] == 0, name='test')
                else:
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) / self.M <= self.u[j, t] + self.v[j, t] + self.v_bar[j, t + 1])
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) >= self.u[j, t])
        
        #給定起火點
        for i in self.N_F:                           
            self.model.addConstr(self.u[i, 0] == 1)
        
        #給定所有節點已經燒的起始狀態
        for i in self.N:                           
            self.model.addConstr(self.v[i, 0] == 0)
    
        #給定所有節點已經保護的起始狀態
        for i in self.N:                            
            self.model.addConstr(self.v_bar[i, 0] == 0)

        #消防員不能去已經被燃燒的節點
        for k in self.K:                             
            for t in self.T:
                for l in range(len(self.A_p)):
                    if self.A_p[l][1] not in self.N_D:
                        if self.A_p[l][0] ==  self.A_p[l][1]:
                            if t + 2 <= self.T_number:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + 1]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                            else:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        elif t + self.tau[self.A_p[l][0], self.A_p[l][1], k] + 1 <= self.T_number:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + self.tau[self.A_p[l][0], self.A_p[l][1], k]]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        else:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
        
        #設定目標式
        self.model.setObjective(gp.quicksum(gp.quicksum(self.u[i, t] for t in self.T) * self.b[i] for i in self.N - self.N_D) + 
                           gp.quicksum(self.epsilon * self.x[i, j, k, t] for (i, j, k, t) in self.x if i != j) +
                           gp.quicksum(self.epsilon * t * self.u_bar[i, k, t] for i in self.N - self.N_D for k in self.K for t in self.T), GRB.MINIMIZE)
        return self.model
        
    def showTextSol(self):
        print("x:")
        for k in self.K:
            print()
            print("消防員%d的路徑" % k)
            temp = [elem for elem in self.x if elem[2] == k]
            for (i, j, k, t) in temp:
                if self.x[i, j, k, t].X > self.epsilon:
                    if i != j:
                        print("在時刻 %d 從node%d 移動到 node%d" % (t, i, j), " ,travel time:", self.tau[i, j, k])
                    else:            
                        if self.u_bar[i,k,t].X == 1:
                            print("在時刻 %d 對node%d進行保護" % (t,i), " ,processing time:", self.process[k][i])
                        else:
                            print("在時刻 %d 在node%d idle" % (t,i))

        print("w:")
        for (i, k, t) in self.w:
            if self.w[i, k, t].X > self.epsilon:
                print("w[%d,%d,%d]" % (i, k, t) , self.w[i, k, t].X)

        print("u:")
        for (i, t) in self.u:
            if self.u[i,t].X > self.epsilon:
                print("u[%d,%d]" % (i,t), self.u[i,t].X)

        print("u_bar:")
        for (i, k, t) in self.u_bar:
            if self.u_bar[i, k, t].X > self.epsilon:
                print("u_bar[%d,%d,%d]" % (i, k, t), self.u_bar[i, k, t].X)

        print("v:")
        for (i, t) in self.v:
            if self.v[i, t].X > self.epsilon:
                print("v[%d,%d]" % (i,t), self.v[i, t].X)

        print("v_bar:")
        for (i, t) in self.v_bar:
            if self.v_bar[i, t].X > self.epsilon:
                print("v_bar[%d,%d]" % (i,t), self.v_bar[i, t].X)
    def writeJson(self):
        data = {}
        data['NODE_POS'] = self.NODE_POS
        data['N'] = list(self.N)
        data['N_D'] = list(self.N_D)
        data['N_F'] = list(self.N_F)
        data['K'] = list(self.K)
        data['A_p'] = list([str(i) for i in self.A_p])
        data['A_f'] = list([str(i) for i in self.A_f])
        data['tau'] = dict((str(i), self.tau[i]) for i in self.tau)
        data['lamb'] = dict((str(i), self.lamb[i]) for i in self.lamb)
        data['T'] = self.T
        data['q'] = self.Q
        data['b'] = self.b
        data['p'] = self.P
        data['h'] = self.H
        
        temp = {}
        for (i, j, k, t) in self.x:
            temp[str((i, j, k, t))] = self.x[i, j, k, t].X
        data['x'] = temp

        temp = {}
        for (i, k, t) in self.w:
            temp[str((i, k, t))] = self.w[i, k, t].X
        data['w'] = temp

        temp = {}
        for (i, t) in self.u:
            temp[str((i, t))] = self.u[i, t].X
        data['u'] = temp

        temp = {}
        for (i, k, t) in self.u_bar:
            temp[str((i, k, t))] = self.u_bar[i, k, t].X
        data['u_bar'] = temp

        temp = {}
        for (i, t) in self.v:
            temp[str((i, t))] = self.v[i, t].X
        data['v']  = temp

        temp = {}
        for (i, t) in self.v_bar:
            temp[str((i, t))] = self.v_bar[i, t].X
        data['v_bar'] = temp
        
        json_data = json.dumps(data)
        with open("data.json", "w") as file:
            file.write(json.dumps(data))

In [3]:
file1 = "G30_fire_route.xlsx"
file2 = "G30_firefighter_route.xlsx"
file3 = "G30_nodeInformation.xlsx"

ff = firefighterProblem(file1, file2, file3)
model = ff.initialize()

#run model and write lp file
model.optimize()
model.write('test.lp')
#print("optimal value : ", model.ObjVal)

ff.showTextSol()
ff.writeJson()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-26
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 55186 rows, 48320 columns and 199674 nonzeros
Model fingerprint: 0x0307accc
Variable types: 0 continuous, 48320 integer (48320 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 42187 rows and 30043 columns
Presolve time: 2.69s
Presolved: 12999 rows, 18277 columns, 55227 nonzeros
Variable types: 0 continuous, 18277 integer (18277 binary)
Found heuristic solution: objective 245.0034000

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8059    9.0919832e+01   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 9.091983e+01, 8059 iterations, 2.23 seconds (1.12 work u

   522   262  131.69357   62  538  155.02920  112.46340  27.5%   514  168s
   574   267  145.03574   70  314  155.02920  113.95788  26.5%   498  172s
   612   262  152.18574   30   87  155.02920  114.21061  26.3%   497  176s
   641   261  153.38043   16   75  155.02920  114.52273  26.1%   502  180s
   705   273  146.77370   28  153  155.02920  114.93824  25.9%   506  189s
   741   264  152.01971   19  262  155.02920  115.65533  25.4%   509  193s
   760   267  148.77873   34  421  155.02920  117.85734  24.0%   525  198s
   793   286     cutoff   29       155.02920  118.05394  23.9%   529  202s
   857   315  121.66935   13  698  155.02920  118.56705  23.5%   517  207s
   911   350  128.19115   22  508  155.02920  118.56705  23.5%   507  211s
   975   354  132.89281   34  505  155.02920  119.03772  23.2%   493  216s
  1035   357  134.77328   37  529  155.02920  120.15471  22.5%   487  239s
  1049   383  140.01425   49  482  155.02920  120.35670  22.4%   488  245s
  1123   393  139.29573  

v[11,57] 1.0
v[11,58] 1.0
v[11,59] 1.0
v[11,60] 1.0
v[11,61] 1.0
v[11,62] 1.0
v[11,63] 1.0
v[11,64] 1.0
v[11,65] 1.0
v[11,66] 1.0
v[11,67] 1.0
v[11,68] 1.0
v[11,69] 1.0
v[11,70] 1.0
v[11,71] 1.0
v[11,72] 1.0
v[11,73] 1.0
v[11,74] 1.0
v[11,75] 1.0
v[11,76] 1.0
v[11,77] 1.0
v[11,78] 1.0
v[11,79] 1.0
v[11,80] 1.0
v[11,81] 1.0
v[11,82] 1.0
v[11,83] 1.0
v[11,84] 1.0
v[11,85] 1.0
v[11,86] 1.0
v[11,87] 1.0
v[11,88] 1.0
v[11,89] 1.0
v[11,90] 1.0
v[11,91] 1.0
v[11,92] 1.0
v[11,93] 1.0
v[11,94] 1.0
v[11,95] 1.0
v[11,96] 1.0
v[11,97] 1.0
v[11,98] 1.0
v[11,99] 1.0
v[11,100] 1.0
v[11,101] 1.0
v[11,102] 1.0
v[11,103] 1.0
v[11,104] 1.0
v[11,105] 1.0
v[11,106] 1.0
v[11,107] 1.0
v[11,108] 1.0
v[11,109] 1.0
v[11,110] 1.0
v[11,111] 1.0
v[11,112] 1.0
v[11,113] 1.0
v[11,114] 1.0
v[11,115] 1.0
v[11,116] 1.0
v[11,117] 1.0
v[11,118] 1.0
v[11,119] 1.0
v[11,120] 1.0
v[11,121] 1.0
v[11,122] 1.0
v[11,123] 1.0
v[11,124] 1.0
v[11,125] 1.0
v[11,126] 1.0
v[11,127] 1.0
v[11,128] 1.0
v[11,129] 1.0
v[11,130] 1.0
v[11,13